# Code for Classification of toric colorable seeds of Picard number $4$
In terms of:
    - Positivity
    - Fan-givingness
    - Projectivity

In [1]:
import SimplicialComplex as sc
import json
import sympy as sp
import numpy as np

def read_file(filename):
    with open(filename, 'rb') as f:
        data = f.readlines()
        data = [x.strip() for x in data]
    return data

### Function for loading all the seeds of a given pair $(m,n)$

In [2]:
def load_seeds(n,m):
    db_path = 'final_results/CSPLS_%d_%d' % (n, m)
    list_facets = [json.loads(facets_bytes) for facets_bytes in read_file(db_path)]
    return [sc.PureSimplicialComplex(facets_set) for facets_set in list_facets]

    
    

### Sympy to Z3py

In [3]:
from z3 import Int, Sqrt 
from sympy.core import Mul, Expr, Add, Pow, Symbol, Number

def sympy_to_z3(sympy_var_list, sympy_exp):
    'convert a sympy expression to a z3 expression. This returns (z3_vars, z3_expression)'

    z3_vars = []
    z3_var_map = {}

    for var in sympy_var_list:
        name = var.name
        z3_var = Int(name)
        z3_var_map[name] = z3_var
        z3_vars.append(z3_var)

    result_exp = _sympy_to_z3_rec(z3_var_map, sympy_exp)

    return z3_vars, result_exp

def _sympy_to_z3_rec(var_map, e):
    'recursive call for sympy_to_z3()'

    rv = None

    if not isinstance(e, Expr):
        raise RuntimeError("Expected sympy Expr: " + repr(e))

    if isinstance(e, Symbol):
        rv = var_map.get(e.name)

        if rv == None:
            raise RuntimeError("No var was corresponds to symbol '" + str(e) + "'")

    elif isinstance(e, Number):
        rv = float(e)
    elif isinstance(e, Mul):
        rv = _sympy_to_z3_rec(var_map, e.args[0])

        for child in e.args[1:]:
            rv *= _sympy_to_z3_rec(var_map, child)
    elif isinstance(e, Add):
        rv = _sympy_to_z3_rec(var_map, e.args[0])

        for child in e.args[1:]:
            rv += _sympy_to_z3_rec(var_map, child)
    elif isinstance(e, Pow):
        term = _sympy_to_z3_rec(var_map, e.args[0])
        exponent = _sympy_to_z3_rec(var_map, e.args[1])

        if exponent == 0.5:
            # sqrt
            rv = Sqrt(term)
        else:
            rv = term**exponent

    if rv == None:
        raise RuntimeError("Type '" + str(type(e)) + "' is not yet implemented for convertion to a z3 expresion. " + \
                            "Subexpression was '" + str(e) + "'.")

    return rv

### Function for testing positivity of a pair $(K,\lambda^{\mathbb{R}})$

In [4]:
from z3 import Solver, sat

def is_positive(K:sc.PureSimplicialComplex,mod_2_IDCM,orientation):
    (m,n) = (K.m,K.n)
    mod_2_char_map = np.zeros((n,m))
    mod_2_char_map[:,:n] = np.eye(n)
    mod_2_char_map[:,n:] = mod_2_IDCM[:,:n].T
    x_vars = sp.symarray('x',(n,m-n))
    R = sp.ZZ.old_poly_ring(x_vars)
    poly_char_map = sp.Matrix(mod_2_char_map)
    for i in range(n):
        for j in range(m):
            if mod_2_char_map[i,j]==1:
                poly_char_map[i,j] = sp.Integer(1)
            else:
                poly_char_map[i,j] = sp.Integer(0)
    poly_char_map[:,n:] += sp.Integer(2)*x_vars
    N = len(K.facets_bin)
    s = Solver()
    for k in range(1,len(K.facets_bin)):
        z3_vars, z3_poly = sympy_to_z3([x_vars[i,j] for i in range(n) for j in range(m-n)],poly_char_map[:,sc.binary_to_face_0(K.facets_bin[k],m)].det().expand()-sp.Integer(orientation[k]))
        s.add(z3_poly==0)
    # for i in range(N):
    #     for j in range(i+1,N):
    #         intersection = sc.binary_to_face_0(K.facets_bin[i]&K.facets_bin[j],m)
    #         if len(intersection)==n-1:
    #             vertices = sc.binary_to_face_0(K.facets_bin[i]^K.facets_bin[j],m)
    #             intersection1 = intersection.copy()
    #             intersection1.append(vertices[0])
    #             intersection2 = intersection.copy()
    #             intersection2.append(vertices[1])
    #             z3_vars, z3_poly = sympy_to_z3([x_vars[i,j] for i in range(n) for j in range(m-n)],(poly_char_map[:,intersection1].det()*poly_char_map[:,intersection2].det()).expand()+sp.Integer(1)) 
    #             s.add(z3_poly==0)
    test = s.check()
    return test==sat


In [5]:
import tqdm
list_non_positive = []
Data = load_seeds(4,8)
counter = 0
for K in tqdm.tqdm(Data):
    seed_is_positive = False
    (m,n) = (K.m,K.n)
    orientation = np.zeros(len(K.facets_bin))
    orientation[0]=1
    for k in range(1,len(K.facets_bin)):
        if orientation[k]!=0:
            continue
        facet_bin_1 = K.facets_bin[k]
        for i in np.where(orientation!=0)[0]:
            facet_bin_2 = K.facets_bin[i]
            vertices = sc.binary_to_face_0(facet_bin_1^facet_bin_2,m)
            if len(vertices)==2:
                v1=vertices[0]
                facet1 = sc.binary_to_face_0(facet_bin_1,m)
                v2=vertices[1]
                facet2 = sc.binary_to_face_0(facet_bin_2,m)
                if v1 not in sc.binary_to_face_0(facet_bin_1,m):
                    v1, v2 = v2, v1
                orientation[k]=(-1)**(facet2.index(v2)+facet1.index(v1)+1)
                break
            
    for mod_2_char_map_bin in sc.IDCM_Garrison_Scott(K):
        mod_2_char_map = sc.char_funct_bin_to_numpy(mod_2_char_map_bin,K.m-K.n)
        seed_is_positive |= is_positive(K,mod_2_char_map,orientation)
        if seed_is_positive:
            break
    if not seed_is_positive:
        list_non_positive.append(K)
    else:
        print(K.facets)
        counter+=1
print("NUmber of positive: ",counter)

  0%|          | 0/21 [00:00<?, ?it/s]

0.0
2 + 2*ToReal(x_3_0)
-2 + -2*ToReal(x_0_0)
-2*ToReal(x_0_0) +
-2*ToReal(x_2_1) +
2*ToReal(x_2_0) +
-4*ToReal(x_0_0)*ToReal(x_2_1) +
4*ToReal(x_0_1)*ToReal(x_2_0)
-2 +
-2*ToReal(x_0_1) +
-2*ToReal(x_1_0) +
2*ToReal(x_1_1) +
-4*ToReal(x_0_1)*ToReal(x_1_0) +
4*ToReal(x_0_0)*ToReal(x_1_1)
-2 + -2*ToReal(x_2_2)
2 + 2*ToReal(x_1_2)
-2 +
-2*ToReal(x_2_2) +
-2*ToReal(x_3_0) +
2*ToReal(x_2_0) +
-4*ToReal(x_2_2)*ToReal(x_3_0) +
4*ToReal(x_2_0)*ToReal(x_3_2)
-2 +
-2*ToReal(x_0_1) +
-2*ToReal(x_2_2) +
2*ToReal(x_0_2) +
-4*ToReal(x_0_1)*ToReal(x_2_2) +
4*ToReal(x_0_2)*ToReal(x_2_1)
2*ToReal(x_0_1) +
2*ToReal(x_1_2) +
-4*ToReal(x_0_2)*ToReal(x_1_1) +
4*ToReal(x_0_1)*ToReal(x_1_2)
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
...
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
... +
